In [ ]:
pip install -U keras-tuner

In [1]:
import tensorflow as tf
import kerastuner as kt
from kerastuner import HyperParameters

In [24]:
#Edit this to point to project root
sep = "\\"
home = f"B:{sep}Dev{sep}GitHub{sep}University{sep}MSc Thesis{sep}StolenVehicleDetector{sep}Machine Learning{sep}OCR{sep}"
local_home = f"{home}"

In [25]:
hyperopt_dir = f"{home}hyperopt{sep}"

In [26]:
def black_box_function(x, y, z):
    value = ( (x ** 2) + (y ** 2) + (z ** 2) )
    return value

In [27]:
def model_builder(hp):

    lr = hp['learning_rate']
    units = hp['units']

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=units, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss='sparse_categorical_crossentropy',
        metrics=['loss'])

    return model

In [28]:
class OCRTuner(kt.Tuner):

    def run_trial(self, trial):

        hp = trial.hyperparameters
        # Hyperparameters can be added anywhere inside `run_trial`.
        # When the first trial is run, they will take on their default values.
        # Afterwards, they will be tuned by the `Oracle`.
        x = hp['x']
        y = hp['y']
        z = hp['z']

        model = self.hypermodel.build(trial.hyperparameters)
        val_loss = black_box_function(x, y, z)
        self.oracle.update_trial(trial.trial_id, {'val_loss': val_loss})

In [29]:
hp = HyperParameters()

# constants
hp.Fixed(name='units', value=5)
hp.Fixed(name='residual_needed', value=True)
# variables
hp.Choice(name='learning_rate', values=[1e-2, 1e-3, 1e-4])
hp.Boolean(name='batchnorm_needed', default=True)
hp.Float(name='dropout', min_value=0, max_value=0.6, step=None, sampling='linear', default=0.2)
hp.Int(name='x', min_value=1, max_value=20, step=1, default=10)
hp.Int(name='y', min_value=1, max_value=20, step=1, default=10)
hp.Int(name='z', min_value=1, max_value=20, step=1, default=10)

10

In [30]:
tuner = OCRTuner(
    oracle=kt.oracles.BayesianOptimization(objective=kt.Objective('val_loss', 'min'), hyperparameters=hp, max_trials=10, num_initial_points=2),
    hypermodel=model_builder,
    directory=hyperopt_dir,
    project_name='OCR tuning'
)

In [31]:
tuner.search_space_summary()

Search space summary
Default search space size: 8
units (Fixed)
{'conditions': [], 'value': 5}
residual_needed (Fixed)
{'conditions': [], 'value': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
batchnorm_needed (Boolean)
{'default': True, 'conditions': []}
dropout (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.0, 'max_value': 0.6, 'step': None, 'sampling': 'linear'}
x (Int)
{'default': 10, 'conditions': [], 'min_value': 1, 'max_value': 20, 'step': 1, 'sampling': None}
y (Int)
{'default': 10, 'conditions': [], 'min_value': 1, 'max_value': 20, 'step': 1, 'sampling': None}
z (Int)
{'default': 10, 'conditions': [], 'min_value': 1, 'max_value': 20, 'step': 1, 'sampling': None}


In [33]:
tuner.search()

INFO:tensorflow:Oracle triggered exit


In [34]:
# Get the optimal hyperparameters
optimal_hyparams = tuner.get_best_hyperparameters(num_trials=1)[0]
print(optimal_hyparams.values)

{'units': 5, 'residual_needed': True, 'learning_rate': 0.01, 'batchnorm_needed': True, 'dropout': 0.6, 'x': 1, 'y': 1, 'z': 1}
